#   **MAPREDUCE CON BOW**
   ----




# Alumno: Huillca Mozo, Bryan
# Codigo: 160329

# ***Instalacion de dependencias para pyspark***

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:13 http://ppa

In [2]:
import os
#Configuracion de variables JavaHome  y Spark_Home
#java_Home
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# Spark_Home
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

!ls

# usamos findspark para configurar Spark en nuestro cuaderno de colab
#  o tambien jupiter si es el caso
import findspark
# inicializacion
findspark.init()
# Usamos PySpark
import pyspark
#Importamos SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

sample_data  spark-2.3.1-bin-hadoop2.7	spark-2.3.1-bin-hadoop2.7.tgz


In [3]:
!pip install pyspark==3.0.1 py4j==0.10.9

     |████████████████████████████████| 204.2 MB 34 kB/s 
     |████████████████████████████████| 198 kB 66.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612246 sha256=e7a1cdadf9c3b4b56bfeda30d0ffdeb5e6ef4eaf7d5b7aafe7d175d83825370f
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


#libreria os para crear archivos temporales y directorios mira el módulo tempfile

In [4]:
# Otras librerias usadas
import os
import pyspark
# Importamos HasingTF: Asigna una secuencia de términos a sus 
# frecuencias de términos utilizando el truco de hash
# IDF: Frecuencia inversa de documentos (IDF)
# Tokenizer: convierte la cadena de entrada a minúsculas 
# y luego la divide por espacios en blanco.
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.sql import SparkSession
import pandas as pd
pd.set_option('display.max_colwidth', 150)

# TEXTOS PARA HALLAR:
---


1.   BOLSA DE PALABRAS
2.   TF
3.   IDF
4.   TIFDF




#Coleccionar palabras de archivo

In [27]:
texto1 =  "Computers have come a long way with the passage of time. Computers will dominate us in the future."
 
texto2 = "programming in haskell is boring, but in pyspark it looks very beautiful"
 


# Creamos la sesion
---

In [29]:
import os
import pyspark
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.sql import SparkSession
import pandas as pd
pd.set_option('display.max_colwidth', 150)

In [30]:
sc = SparkSession\
        .builder\
        .appName("TfIdf ")\
        .getOrCreate()

In [31]:
documents = sc.createDataFrame([
        (1, texto1),
        (2, texto2)
    ], ["doc_id", "doc_text"])

# MOSTRAMOS LOS DOS TEXTOS
---

In [32]:
documents.toPandas()

,doc_id,doc_text
0,1,Computers have come a long way with the passage of time. Computers will dominate us in the future.
1,2,"programming in haskell is boring, but in pyspark it looks very beautiful"


# Volvemos nuestro dataframe data que contiene nuestros
# textos y lo convertimos a rdd
---

In [34]:
data = (documents.rdd
       .map(lambda x: (x.doc_id,x.doc_text.lower().replace('.','')
                       .replace(',','').replace(';','').
                       replace('?','').replace('!','').
                       replace(':','').split(" ")))
       .toDF()
       .withColumnRenamed("_1","doc_id")
       .withColumnRenamed("_2","features"))

# MODULO BAG OF WORDS
---

In [35]:
def BagOfWords(documents):
    t=documents.flatMap(lambda x: [(i.lower(),len(i)) for i in x[1].split()])
    MapT=t.groupByKey()
    bow=mapeo.map(lambda x: x[0])
    bow = bow.filter(lambda x: len(x) > 2)
    
    return bow

In [36]:
data.toPandas()

,doc_id,features
0,1,"[computers, have, come, a, long, way, with, the, passage, of, time, computers, will, dominate, us, in, the, future]"
1,2,"[programming, in, haskell, is, boring, but, in, pyspark, it, looks, very, beautiful]"


# Asignamos secuencia de términos a sus frecuencias de 
# términos utilizando el truco de hash.


# Esto para calcular el valor del código hash para el objeto de término. Dado que se usa un módulo simple para transformar la función hash en un índice de columna.

# VECTORIZAMOS - TF
---


In [45]:
htf= HashingTF(inputCol="features", outputCol="tf")#numFeatures=20)
tf = htf.transform(data)
#tf.show(truncate=False)
tf.head(2)

[Row(doc_id=1, features=['computers', 'have', 'come', 'a', 'long', 'way', 'with', 'the', 'passage', 'of', 'time', 'computers', 'will', 'dominate', 'us', 'in', 'the', 'future'], tf=SparseVector(262144, {9639: 1.0, 21872: 1.0, 52918: 2.0, 89356: 1.0, 103838: 2.0, 121517: 1.0, 126466: 1.0, 175390: 1.0, 199255: 1.0, 204380: 1.0, 206312: 1.0, 222453: 1.0, 227410: 1.0, 253475: 1.0, 255766: 1.0, 258647: 1.0})),
 Row(doc_id=2, features=['programming', 'in', 'haskell', 'is', 'boring', 'but', 'in', 'pyspark', 'it', 'looks', 'very', 'beautiful'], tf=SparseVector(262144, {1998: 1.0, 10645: 1.0, 15889: 1.0, 34210: 1.0, 53604: 1.0, 60007: 1.0, 86175: 1.0, 110693: 1.0, 189683: 1.0, 210040: 1.0, 222453: 2.0}))]

# TF
---

In [38]:
tf.toPandas()

,doc_id,features,tf
0,1,"[computers, have, come, a, long, way, with, the, passage, of, time, computers, will, dominate, us, in, the, future]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,2,"[programming, in, haskell, is, boring, but, in, pyspark, it, looks, very, beautiful]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


# IDF
---

#Frecuencia inversa de documentos 

#admite el filtrado de términos que no aparecen en un número mínimo de documentos

In [39]:
idf = IDF(inputCol="tf",outputCol="idf")
tfidf = idf.fit(tf).transform(tf)
tfidf.head(2)

[Row(doc_id=1, features=['computers', 'have', 'come', 'a', 'long', 'way', 'with', 'the', 'passage', 'of', 'time', 'computers', 'will', 'dominate', 'us', 'in', 'the', 'future'], tf=SparseVector(262144, {9639: 1.0, 21872: 1.0, 52918: 2.0, 89356: 1.0, 103838: 2.0, 121517: 1.0, 126466: 1.0, 175390: 1.0, 199255: 1.0, 204380: 1.0, 206312: 1.0, 222453: 1.0, 227410: 1.0, 253475: 1.0, 255766: 1.0, 258647: 1.0}), idf=SparseVector(262144, {9639: 0.4055, 21872: 0.4055, 52918: 0.8109, 89356: 0.4055, 103838: 0.8109, 121517: 0.4055, 126466: 0.4055, 175390: 0.4055, 199255: 0.4055, 204380: 0.4055, 206312: 0.4055, 222453: 0.0, 227410: 0.4055, 253475: 0.4055, 255766: 0.4055, 258647: 0.4055})),
 Row(doc_id=2, features=['programming', 'in', 'haskell', 'is', 'boring', 'but', 'in', 'pyspark', 'it', 'looks', 'very', 'beautiful'], tf=SparseVector(262144, {1998: 1.0, 10645: 1.0, 15889: 1.0, 34210: 1.0, 53604: 1.0, 60007: 1.0, 86175: 1.0, 110693: 1.0, 189683: 1.0, 210040: 1.0, 222453: 2.0}), idf=SparseVector(262

# SELECCION IDF
---

In [40]:
test=tfidf.select('tf').show()
test

+--------------------+
|                  tf|
+--------------------+
|(262144,[9639,218...|
|(262144,[1998,106...|
+--------------------+



In [41]:
df=tfidf.toPandas()
df


,doc_id,features,tf,idf
0,1,"[computers, have, come, a, long, way, with, the, passage, of, time, computers, will, dominate, us, in, the, future]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,2,"[programming, in, haskell, is, boring, but, in, pyspark, it, looks, very, beautiful]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


# TFIDF
---

In [ ]:
res = tfidf.rdd.map(lambda x:(x.doc_id,x.features,x.tf,x.idf,(None if x.idf is None else x.idf.values.sum())))
for r in res.take(10):
    print (r)

# FUNCIONES DE USUARIO
---

# UDF: para definir nuevos métodos SQL que operan sobre las columnas de un DataFrame.

#COMPARACIONES ENTRE PALABRAS (FEATURES)
# TF IDF  Y TIFDF

In [43]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType


sum_ = udf(lambda v: float(v.values.sum()), DoubleType())
tfidf.withColumn("tfidf", sum_("idf")).show()

+------+--------------------+--------------------+--------------------+-----------------+
|doc_id|            features|                  tf|                 idf|            tfidf|
+------+--------------------+--------------------+--------------------+-----------------+
|     1|[computers, have,...|(262144,[9639,218...|(262144,[9639,218...|6.892906837838795|
|     2|[programming, in,...|(262144,[1998,106...|(262144,[1998,106...|4.054651081081644|
+------+--------------------+--------------------+--------------------+-----------------+

